In [3]:
def missing_impute(df):

    #Preparing a list of DataFrame columns having missing values

    missing = []
    for miss in df.columns:
        if(df[miss].isnull().any()):
            missing.append(miss)

    #list of correlation values of all features with the target
    for col in missing:
        null = list(df.loc[df[col].isnull(),:].index)
        corrmat = df.corr()
        l1 = []
        for feat in corrmat.columns:
              if(feat==col):
                for val in corrmat[feat].values:
                    l1.append(val)
                l1.sort()

      #Filtering out the highly correlated features with target
        count = 0
        corval = 0.9
        while(1):
            for i in l1:
                if(i>=corval):
                    count+=1
                if(count>=10):
                    correlated = corrmat[abs(corrmat[col])>=corval][col]
                    break
                else:
                    corval-=0.1
                    continue
      # Checking multicollinearity, write code here
      

         #Using highly correlated features to predict the target(missing value)
    x1 = df.loc[df[col].notnull(),:]
    for feature in x1.columns:
          if(feature not in correlated.index):
                x = x1.drop(feature,axis=1)
    #Predicting using linear regression
    Y_train = x[col]
    #print(Y_train.shape,X_train.shape)
    x = x.drop(col,axis=1)
    X_train = x.dropna(axis=1)
    #print(X_train.columns)
    model = LinearRegression()
    #print(X_train.shape)
    model.fit(X_train,Y_train)
    Pred1 = model.predict(X_train)
    x2 = df.loc[df[col].isnull(),:]
    for feature in x2.columns:                #Considering only those features which are in the X_train
        if(feature not in X_train.columns):
              x2.drop(feature,axis=1,inplace=True)
      #print(x2.shape)
    x2.dropna(axis=1,inplace=True)            
    for i in X_train.columns:
        if(i not in x2.columns):              #Using only those non-null columns and replacing null columns with all 0's
              x2[i] = np.zeros(x2.shape[0])
    #print(x2.shape)
    y_pred = model.predict(x2)
    #print(model.intercept_,model.coef_)      # Prints the slope and intercept 
    print('R2 score is :',model.score(x2,y_pred),'\n RMSE is :',mean_squared_error(Y_train,Pred1))
      
      
    imp = pd.Series(y_pred)
    #feature = list(df[col])
    #df = df.drop(col,axis=1)
    ser = df.loc[:,col].copy()

    # Create dataframe of predicted values 
    impdf = pd.DataFrame(data=imp)
    impdf['index'] = null                     # Store index of null values 

    # Create dataframe of feature containing missing values
    serdf = pd.DataFrame(ser)
    serdf[col].fillna(value=0.0,inplace=True)
    serdf = serdf.reset_index()

    # Merge both dataframes and restore 
    data = pd.merge(serdf,impdf,left_on='index',right_on='index',how='outer')
    data = data.fillna(value = 0.0)
    data[col] = data[col] + data[0]
    df[col] = data[col].copy()